In [9]:
import time
def writetime():
    st = time.time()
    with open("time.log", "a+") as f:
        f.write(str(st) + "\n")
# Fit the model
from mycode.test_20_tables import fetch_data_from_sqlite, Metadata as sdvMetadata
from sdgx.data_connectors.csv_connector import CsvConnector
from sdgx.models.ml.single_table.ctgan import CTGANSynthesizerModel
from sdgx.synthesizer import Synthesizer
# from sdgx.utils import download_demo_data
metadata, tables = fetch_data_from_sqlite(path='./mycode/data_sqlite.db')
metadata = sdvMetadata(metadata)
metadata.get_tables()
import pandas as pd
result_table = tables["BookLoan"]
x_table = ["Book", "Library", "Student", "Enrollment", "Submission"]
x_key = ['book_id', "library_id", "student_id", "student_id","student_id"]
x_how = ['outer' for i in range(5)]
print(f"{len(result_table)=}")
for i, x_table_name in enumerate(x_table):
    result_table = pd.merge(result_table, tables[x_table_name], on=x_key[i], how=x_how[i])
    print(f"{len(result_table)=}")
print(len(result_table.columns))
result_table


len(result_table)=10000
len(result_table)=10000
len(result_table)=10000
len(result_table)=13684
len(result_table)=18813
len(result_table)=26838
23


,loan_id,book_id,student_id,loan_date,return_date,title,author,isbn,library_id,name_x,...,date_of_birth,major,enrollment_id,course_id,semester,grade_x,submission_id,assignment_id,submission_date,grade_y
0,1.0,1.0,5068,2024-07-19,2025-07-30,Model bank.,Ricky Brown,978-1-4017-1762-9,1.0,Wilson Group Library,...,2003-01-27,1262,5758.0,9618.0,Fall 2022,C,2462.0,9937.0,2022-10-18 02:02:31,B
1,1.0,1.0,5068,2024-07-19,2025-07-30,Model bank.,Ricky Brown,978-1-4017-1762-9,1.0,Wilson Group Library,...,2003-01-27,1262,5758.0,9618.0,Fall 2022,C,3984.0,4403.0,2022-11-15 20:13:23,I
2,1.0,1.0,5068,2024-07-19,2025-07-30,Model bank.,Ricky Brown,978-1-4017-1762-9,1.0,Wilson Group Library,...,2003-01-27,1262,5758.0,9618.0,Fall 2022,C,7851.0,6252.0,2022-07-31 08:44:12,A
3,1.0,1.0,5068,2024-07-19,2025-07-30,Model bank.,Ricky Brown,978-1-4017-1762-9,1.0,Wilson Group Library,...,2003-01-27,1262,9382.0,5777.0,Summer 2023,A,2462.0,9937.0,2022-10-18 02:02:31,B
4,1.0,1.0,5068,2024-07-19,2025-07-30,Model bank.,Ricky Brown,978-1-4017-1762-9,1.0,Wilson Group Library,...,2003-01-27,1262,9382.0,5777.0,Summer 2023,A,3984.0,4403.0,2022-11-15 20:13:23,I
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26833,NaN,NaN,9994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1994-05-08,6802,6944.0,1428.0,Fall 2021,C,7756.0,1543.0,2022-08-11 03:17:57,B
26834,NaN,NaN,9996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2001-10-16,8561,NaN,NaN,NaN,NaN,1699.0,6848.0,2022-06-15 17:55:35,F
26835,NaN,NaN,9996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2001-10-16,8561,NaN,NaN,NaN,NaN,6740.0,7011.0,2022-05-25 09:43:43,F
26836,NaN,NaN,9996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2001-10-16,8561,NaN,NaN,NaN,NaN,9676.0,2748.0,2022-12-26 02:38:34,A


In [4]:
result_table.columns
result_table.head()

,loan_id,book_id,student_id,loan_date,return_date,title,author,isbn,library_id,name_x,...,date_of_birth,major,enrollment_id,course_id,semester,grade_x,submission_id,assignment_id,submission_date,grade_y
0,1,1,5068,2024-07-19,2025-07-30,Model bank.,Ricky Brown,978-1-4017-1762-9,1,Wilson Group Library,...,2003-01-27,1262,5758,9618,Fall 2022,C,2462,9937,2022-10-18 02:02:31,B
1,1,1,5068,2024-07-19,2025-07-30,Model bank.,Ricky Brown,978-1-4017-1762-9,1,Wilson Group Library,...,2003-01-27,1262,5758,9618,Fall 2022,C,3984,4403,2022-11-15 20:13:23,I
2,1,1,5068,2024-07-19,2025-07-30,Model bank.,Ricky Brown,978-1-4017-1762-9,1,Wilson Group Library,...,2003-01-27,1262,5758,9618,Fall 2022,C,7851,6252,2022-07-31 08:44:12,A
3,1,1,5068,2024-07-19,2025-07-30,Model bank.,Ricky Brown,978-1-4017-1762-9,1,Wilson Group Library,...,2003-01-27,1262,9382,5777,Summer 2023,A,2462,9937,2022-10-18 02:02:31,B
4,1,1,5068,2024-07-19,2025-07-30,Model bank.,Ricky Brown,978-1-4017-1762-9,1,Wilson Group Library,...,2003-01-27,1262,9382,5777,Summer 2023,A,3984,4403,2022-11-15 20:13:23,I


In [2]:
result_table = result_table.drop(columns=["title", "author", "name_x", "semester", "location", "name_y", "email"])

In [3]:
result_table = result_table.drop(columns=["isbn"])

In [ ]:
result_table = result_table.drop(columns=["grade_x", "grade_y"])

In [4]:
result_table.head()

,loan_id,book_id,student_id,loan_date,return_date,library_id,date_of_birth,major,enrollment_id,course_id,grade_x,submission_id,assignment_id,submission_date,grade_y
0,1,1,5068,2024-07-19,2025-07-30,1,2003-01-27,1262,5758,9618,C,2462,9937,2022-10-18 02:02:31,B
1,1,1,5068,2024-07-19,2025-07-30,1,2003-01-27,1262,5758,9618,C,3984,4403,2022-11-15 20:13:23,I
2,1,1,5068,2024-07-19,2025-07-30,1,2003-01-27,1262,5758,9618,C,7851,6252,2022-07-31 08:44:12,A
3,1,1,5068,2024-07-19,2025-07-30,1,2003-01-27,1262,9382,5777,A,2462,9937,2022-10-18 02:02:31,B
4,1,1,5068,2024-07-19,2025-07-30,1,2003-01-27,1262,9382,5777,A,3984,4403,2022-11-15 20:13:23,I


In [10]:
from sdgx.data_loader import DataLoader
from sdgx.data_models.metadata import Metadata

result_table.to_csv("test_10k_noncat.csv")
dataset_csv = "test_10k_noncat.csv"
data_connector = CsvConnector(path=dataset_csv)
data_loader = DataLoader(data_connector)
loan_metadata = Metadata.from_dataloader(data_loader)

loan_metadata.discrete_columns

2024-09-27 12:48:33.450 | INFO     | sdgx.data_models.metadata:from_dataloader:291 - Inspecting metadata...
2024-09-27 12:48:40.630 | INFO     | sdgx.data_models.metadata:update_primary_key:500 - Primary Key updated: {'Unnamed: 0'}.


{'author',
 'date_of_birth',
 'email',
 'grade_x',
 'grade_y',
 'isbn',
 'loan_date',
 'location',
 'name_x',
 'name_y',
 'return_date',
 'semester',
 'submission_date',
 'title'}

In [19]:
loan_metadata.datetime_columns
loan_metadata.column_list.__len__()

16

In [11]:


ctgan = CTGANSynthesizerModel(
        epochs=1,
        batch_size=1000,
        # device="cpu"
    )
# Initialize synthesizer, use CTGAN model
synthesizer = Synthesizer(
    model=ctgan,  # For quick demo
    data_connector=data_connector,
)


writetime()
synthesizer.fit()
writetime()
#synthesizer.save("test_model_10k_id10")

2024-09-27 12:48:42.472 | INFO     | sdgx.synthesizer:__init__:106 - Using data processors: ['nonvaluetransformer', 'outliertransformer', 'emailgenerator', 'chnpiigenerator', 'intvalueformatter', 'datetimeformatter', 'constvaluetransformer', 'emptytransformer', 'columnordertransformer']
2024-09-27 12:48:42.475 | INFO     | sdgx.data_models.metadata:from_dataloader:291 - Inspecting metadata...
2024-09-27 12:48:49.968 | INFO     | sdgx.data_models.metadata:update_primary_key:500 - Primary Key updated: {'Unnamed: 0'}.
2024-09-27 12:48:49.970 | INFO     | sdgx.synthesizer:fit:296 - Fitting data processors...


{'primary_keys': {'Unnamed: 0'}, 'column_list': ['Unnamed: 0', 'loan_id', 'book_id', 'student_id', 'loan_date', 'return_date', 'title', 'author', 'isbn', 'library_id', 'name_x', 'location', 'name_y', 'email', 'date_of_birth', 'major', 'enrollment_id', 'course_id', 'semester', 'grade_x', 'submission_id', 'assignment_id', 'submission_date', 'grade_y'], 'column_inspect_level': {'id_columns': 20, 'china_mainland_address_columns': 30, 'discrete_columns': 10, 'datetime_columns': 20, 'china_mainland_postcode_columns': 20, 'chinese_name_columns': 40, 'english_name_columns': 40, 'unified_social_credit_code_columns': 30, 'bool_columns': 10, 'int_columns': 10, 'float_columns': 10, 'email_columns': 30, 'chinese_company_name_columns': 40, 'empty_columns': 90, 'china_mainland_mobile_phone_columns': 30, 'china_mainland_id_columns': 30, 'const_columns': 80}, 'pii_columns': {'author', 'email', 'name_y', 'title'}, 'id_columns': {'Unnamed: 0'}, 'int_columns': {'student_id', 'major', 'book_id', 'Unnamed: 

2024-09-27 12:48:50.355 | INFO     | sdgx.data_processors.transformers.nan:fit:71 - NonValueTransformer Fitted.
2024-09-27 12:48:50.356 | INFO     | sdgx.data_processors.transformers.nan:fit:87 - NonValueTransformer get int columns: {'student_id', 'major', 'book_id', 'loan_id', 'assignment_id', 'enrollment_id', 'submission_id', 'library_id', 'course_id'}.
2024-09-27 12:48:50.357 | INFO     | sdgx.data_processors.transformers.nan:fit:96 - NonValueTransformer get float columns: set().
2024-09-27 12:48:50.359 | INFO     | sdgx.data_processors.transformers.nan:fit:101 - NonValueTransformer get column list from metadata: ['Unnamed: 0', 'loan_id', 'book_id', 'student_id', 'loan_date', 'return_date', 'title', 'author', 'isbn', 'library_id', 'name_x', 'location', 'name_y', 'email', 'date_of_birth', 'major', 'enrollment_id', 'course_id', 'semester', 'grade_x', 'submission_id', 'assignment_id', 'submission_date', 'grade_y'].
2024-09-27 12:48:50.360 | INFO     | sdgx.data_processors.transformers.

  0%|          | 0/26 [00:00<?, ?it/s]

2024-09-27 12:49:23.709 | INFO     | sdgx.models.ml.single_table.ctgan:_fit:382 - Epoch 1, Loss G:  2.0077, Loss D: -18125.3633, Time:  3.0591
2024-09-27 12:49:23.712 | INFO     | sdgx.models.ml.single_table.ctgan:fit:225 - CTGAN training finished.
2024-09-27 12:49:23.713 | INFO     | sdgx.synthesizer:fit:333 - Model fit... Finished


In [12]:
data_res = synthesizer.sample(1000)
data_res

2024-09-27 12:49:27.215 | INFO     | sdgx.synthesizer:sample:357 - Sampling...


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

2024-09-27 12:49:27.974 | INFO     | sdgx.data_processors.transformers.nan:reverse_convert:155 - Data reverse-converted by NonValueTransformer.
2024-09-27 12:49:27.978 | INFO     | sdgx.data_processors.transformers.outlier:reverse_convert:120 - Data reverse-converted by OutlierTransformer (No Action).
2024-09-27 12:49:28.527 | INFO     | sdgx.data_processors.formatters.int:reverse_convert:64 - Data reverse-converted by IntValueFormatter.
2024-09-27 12:49:28.530 | INFO     | sdgx.data_processors.formatters.datetime:reverse_convert:162 - Data reverse-converted by DatetimeFormatter (No datetime columns).
2024-09-27 12:49:28.531 | INFO     | sdgx.data_processors.transformers.const:reverse_convert:103 - Data reverse-converted by ConstValueTransformer.
2024-09-27 12:49:28.531 | INFO     | sdgx.data_processors.transformers.empty:reverse_convert:98 - Data reverse-converted by EmptyTransformer.
2024-09-27 12:49:28.534 | INFO     | sdgx.data_processors.transformers.column_order:reverse_convert:6

,Unnamed: 0,loan_id,book_id,student_id,loan_date,return_date,title,author,isbn,library_id,...,date_of_birth,major,enrollment_id,course_id,semester,grade_x,submission_id,assignment_id,submission_date,grade_y
0,18232,5725,6674,8088,2024-03-24,2025-07-16,Who skin.,Tonya Williams,978-1-908267-43-6,5634,...,2006-05-19,1106,74,9185,Summer 2023,A,1913,9849,2022-11-15 20:13:23,I
1,15877,5704,10399,8631,2024-03-24,2025-07-16,Who skin.,Tonya Williams,978-1-908267-43-6,4812,...,2006-05-19,1707,196,-109,Summer 2023,A,1408,-113,2022-11-15 20:13:23,I
2,20782,6269,9437,1822,2024-03-24,2025-07-16,Who skin.,Tonya Williams,978-1-908267-43-6,1521,...,2006-05-19,6958,9017,2954,Summer 2023,A,8632,6110,2022-11-15 20:13:23,I
3,18823,10382,3472,3356,2024-03-24,2025-07-16,Who skin.,Tonya Williams,978-1-908267-43-6,1271,...,2006-05-19,7403,7120,1806,Summer 2023,A,1460,5321,2022-11-15 20:13:23,I
4,3749,10398,5584,4934,2024-03-24,2025-07-16,Who skin.,Tonya Williams,978-1-908267-43-6,2834,...,2006-05-19,5900,-164,9928,Summer 2023,A,7151,2291,2022-11-15 20:13:23,I
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,27539,6387,10754,7467,2024-03-24,2025-07-16,Who skin.,Tonya Williams,978-1-908267-43-6,9428,...,2006-05-19,7841,8007,10774,Summer 2023,A,147,8879,2022-11-15 20:13:23,I
996,4588,9417,10092,8160,2024-03-24,2025-07-16,Who skin.,Tonya Williams,978-1-908267-43-6,9559,...,2006-05-19,1883,7671,2164,Summer 2023,A,6377,5311,2022-11-15 20:13:23,I
997,14700,8181,9316,5340,2024-03-24,2025-07-16,Who skin.,Tonya Williams,978-1-908267-43-6,5601,...,2006-05-19,9127,8573,10287,Summer 2023,A,10892,1420,2022-11-15 20:13:23,I
998,27681,6211,2982,5007,2024-03-24,2025-07-16,Who skin.,Tonya Williams,978-1-908267-43-6,5963,...,2006-05-19,9730,917,10122,Summer 2023,A,4677,99,2022-11-15 20:13:23,I
